In [1]:
import random as rd

In [2]:
class Node():
    adjacent = []
    nearest_manhole = None
    def __init__(self, name, status=0, samplable=False):
        self.name = name
        self.status = status
        self.samplable = samplable
    def __str__(self):
        return str(self.name) + " | STATUS: " + str(self.status) + " | ADJ: " + str(self.adjacent)

In [3]:
test = Node(0)
print(test)
test2 = Node(2, status=1)
print(test2)

0 | STATUS: 0 | ADJ: []
2 | STATUS: 1 | ADJ: []


In [4]:
def build_SIR_net(file_in):
    nodes = []
    counter = 0
    with open(file_in) as f:
        while(True):
            line = f.readline()
        
            # check for end of file
            if len(line) == 0 or line == "\n":
                break
        
            new = Node(counter)
            split = line.split()
            for i in range(len(split)):
                split[i] = int(split[i])
            new.adjacent = split
            nodes.append(new)
            counter += 1
        
    return nodes

In [5]:
net = build_SIR_net("sample.txt")
print(net)

[<__main__.Node object at 0x00000160F2622190>, <__main__.Node object at 0x00000160F261D2B0>, <__main__.Node object at 0x00000160F261D250>, <__main__.Node object at 0x00000160F2617A00>, <__main__.Node object at 0x00000160F26176D0>]


In [6]:
for i in net:
    print(i)

0 | STATUS: 0 | ADJ: [1, 4]
1 | STATUS: 0 | ADJ: [2, 3]
2 | STATUS: 0 | ADJ: [1, 4]
3 | STATUS: 0 | ADJ: [1, 4]
4 | STATUS: 0 | ADJ: [0, 2]


In [7]:
def simple_simulate(net, rates, time):
    t = 0
    
    infected = []
    recovered = []
    samples = []
    for i in net:
        if i.status == 1:
            infected.append(i)
        elif i.status == 2:
            recovered.append(i)
    
    while(t < time):
        t += 1
        
        new_infected = []
        for i in infected:
            
            # infecting others
            for j in i.adjacent:
                j = net[j]
                if j not in infected and j not in recovered and j not in new_infected:
                    # there is a chance to get infected
                    chance = rd.random()
                    if chance < rates[0]:
                        new_infected.append(j)
            
            # recovering
            chance = rd.random()
            if chance < rates[1]:
                infected.remove(i)
                recovered.append(i)
                
        for i in new_infected:
            infected.append(i)
                
        print("TIME ---------------------------------", t)
        inf_string = ""
        for i in infected:
            inf_string += str(i.name) + " "
        rec_string = ""
        for i in recovered:
            rec_string += str(i.name) + " "
        print("INFECTED:", inf_string, "RECOVERED:", rec_string)
        # print("RECOVERED:", recovered)
        # print("VIABLE SAMPLE LOCATIONS:", samples, "\n")
        
    
    return infected, recovered

In [8]:
net = build_SIR_net("sample.txt")
net[3].status = 1
for i in net:
    print(i)
simple_simulate(net, [0.5, 0.2], 10)

0 | STATUS: 0 | ADJ: [1, 4]
1 | STATUS: 0 | ADJ: [2, 3]
2 | STATUS: 0 | ADJ: [1, 4]
3 | STATUS: 1 | ADJ: [1, 4]
4 | STATUS: 0 | ADJ: [0, 2]
TIME --------------------------------- 1
INFECTED: 3 1 4  RECOVERED: 
TIME --------------------------------- 2
INFECTED: 3 4 2  RECOVERED: 1 
TIME --------------------------------- 3
INFECTED: 4 2  RECOVERED: 1 3 
TIME --------------------------------- 4
INFECTED: 2  RECOVERED: 1 3 4 
TIME --------------------------------- 5
INFECTED: 2  RECOVERED: 1 3 4 
TIME --------------------------------- 6
INFECTED:  RECOVERED: 1 3 4 2 
TIME --------------------------------- 7
INFECTED:  RECOVERED: 1 3 4 2 
TIME --------------------------------- 8
INFECTED:  RECOVERED: 1 3 4 2 
TIME --------------------------------- 9
INFECTED:  RECOVERED: 1 3 4 2 
TIME --------------------------------- 10
INFECTED:  RECOVERED: 1 3 4 2 


([],
  <__main__.Node at 0x160f2659910>])

In [12]:
def build_sewer_net(file_in):
    nodes = []
    counter = 0
    with open(file_in) as f:
        while(True):
            line = f.readline()
        
            # check for end of SIR layer
            if line == "\n":
                break
        
            new = Node(counter)
            split = line.split()
            for i in range(len(split)):
                split[i] = int(split[i])
            new.adjacent = split
            nodes.append(new)
            counter += 1
            
        # sewer time
        sewer = {}
        while(True):
            line = f.readline()
            
            # check for end of manhole assignment
            if line == "\n":
                break
                
            split = line.split()
            name = ""
            for i in range(len(split)):
                name += split[i] + "-"
                split[i] = int(split[i])
            name = name[:-1]
            new = Node(name)
            new.adjacent = split
            new.samplable = True
            sewer[name] = new
            
            for i in split:
                nodes[i].nearest_manhole = name
                
        # connecting manholes
        while(True):
            line = f.readline()
            
            # check for end of file
            if len(line) == 0:
                break
                
            split = line.split()
            if split[1] != 'end':
                sewer[split[0]].nearest_manhole = split[1]
            else:
                end = Node('end')
                end.samplable = True
                sewer['end'] = end
                sewer[split[0]].nearest_manhole = 'end'
        
    return nodes, sewer

In [14]:
net, sewer = build_sewer_net("sample.txt")
net[3].status = 1
for i in net:
    print(i)
    print(i.nearest_manhole)
for i in sewer.keys():
    print(sewer[i])
    print(sewer[i].nearest_manhole)
    
print(sewer)

0 | STATUS: 0 | ADJ: [1, 4]
0-4
1 | STATUS: 0 | ADJ: [2, 3]
1-2-3
2 | STATUS: 0 | ADJ: [1, 4]
1-2-3
3 | STATUS: 1 | ADJ: [1, 4]
1-2-3
4 | STATUS: 0 | ADJ: [0, 2]
0-4
0-4 | STATUS: 0 | ADJ: [0, 4]
1-2-3
1-2-3 | STATUS: 0 | ADJ: [1, 2, 3]
end
end | STATUS: 0 | ADJ: []
None
{'0-4': <__main__.Node object at 0x00000160F2686040>, '1-2-3': <__main__.Node object at 0x00000160F26860A0>, 'end': <__main__.Node object at 0x00000160F2686100>}


In [26]:
def sample_simulate(net, sewer, rates, time):
    t = 0
    
    infected = []
    recovered = []
    samples = []
    for i in net:
        if i.status == 1:
            infected.append(i)
        elif i.status == 2:
            recovered.append(i)
            
    timers = {}
    for i in sewer.keys():
        timers[i] = 99999999999999
    
    while(t < time):
        t += 1
        
        new_infected = []
        for i in infected:
            
            # infecting others
            for j in i.adjacent:
                j = net[j]
                if j not in infected and j not in recovered and j not in new_infected:
                    # there is a chance to get infected
                    chance = rd.random()
                    if chance < rates[0]:
                        new_infected.append(j)
            
            # recovering
            chance = rd.random()
            if chance < rates[1]:
                infected.remove(i)
                recovered.append(i)
                
        for i in new_infected:
            infected.append(i)
            
        # sewer propagation
        # sewer_nxt = []
        for i in sewer.keys():
            if sewer[i].status == 1:
                if sewer[i].nearest_manhole != None:
                    nxt = sewer[i].nearest_manhole[0]
                    weight = sewer[i].nearest_manhole[1]
                    # sewer_nxt.append(nxt)
                    timers[i] = min(timers[i], t + weight)
                # sewer[i].status = 0
                
        for i in timers.keys():
            if timers[i] == t:
                sewer[i].status = 1
        # for i in sewer_nxt:
        #     sewer[i].status = 1
        # for i in infected:
        #     sewer[i.nearest_manhole].status = 1
                
        print("TIME ---------------------------------", t)
        inf_string = ""
        for i in infected:
            inf_string += str(i.name) + " "
        rec_string = ""
        for i in recovered:
            rec_string += str(i.name) + " "
        print("INFECTED:", inf_string, "RECOVERED:", rec_string)
        sample_string = ""
        for i in sewer.keys():
            if sewer[i].status == 1:
                sample_string += i + " "
        print("VIABLE SAMPLE LOCATIONS:", sample_string)
        
    
    return infected, recovered

In [27]:
net, sewer = build_sewer_net("sample.txt")
net[3].status = 1
for i in net:
    print(i)
for i in sewer.keys():
    print(sewer[i])
    
sample_simulate(net, sewer, [0.5, 0.2], 10)

0 | STATUS: 0 | ADJ: [1, 4]
1 | STATUS: 0 | ADJ: [2, 3]
2 | STATUS: 0 | ADJ: [1, 4]
3 | STATUS: 1 | ADJ: [1, 4]
4 | STATUS: 0 | ADJ: [0, 2]
0-4 | STATUS: 0 | ADJ: [0, 4]
1-2-3 | STATUS: 0 | ADJ: [1, 2, 3]
end | STATUS: 0 | ADJ: []
TIME --------------------------------- 1
INFECTED: 1 4  RECOVERED: 3 
VIABLE SAMPLE LOCATIONS: 0-4 1-2-3 
TIME --------------------------------- 2
INFECTED: 1 4 2 0  RECOVERED: 3 
VIABLE SAMPLE LOCATIONS: 0-4 1-2-3 end 
TIME --------------------------------- 3
INFECTED: 1 4 2  RECOVERED: 3 0 
VIABLE SAMPLE LOCATIONS: 0-4 1-2-3 end 
TIME --------------------------------- 4
INFECTED: 1 4  RECOVERED: 3 0 2 
VIABLE SAMPLE LOCATIONS: 0-4 1-2-3 end 
TIME --------------------------------- 5
INFECTED: 4  RECOVERED: 3 0 2 1 
VIABLE SAMPLE LOCATIONS: 0-4 1-2-3 end 
TIME --------------------------------- 6
INFECTED: 4  RECOVERED: 3 0 2 1 
VIABLE SAMPLE LOCATIONS: 0-4 1-2-3 end 
TIME --------------------------------- 7
INFECTED:  RECOVERED: 3 0 2 1 4 
VIABLE SAMPLE LOCAT

([],
  <__main__.Node at 0x160f26591c0>])

In [29]:
net, sewer = build_sewer_net("sample2.txt")
net[3].status = 1
net[6].status = 1
for i in net:
    print(i)
for i in sewer.keys():
    print(sewer[i])
    # print(sewer[i].nearest_manhole)
    
sample_simulate(net, sewer, [0.1, 0.08], 25)

0 | STATUS: 0 | ADJ: [1, 2]
1 | STATUS: 0 | ADJ: [0, 3]
2 | STATUS: 0 | ADJ: [0, 3]
3 | STATUS: 1 | ADJ: [1, 2]
4 | STATUS: 0 | ADJ: [5]
5 | STATUS: 0 | ADJ: [4, 6]
6 | STATUS: 1 | ADJ: [7, 8]
7 | STATUS: 0 | ADJ: [6, 10]
8 | STATUS: 0 | ADJ: [6, 9]
9 | STATUS: 0 | ADJ: [8, 10]
10 | STATUS: 0 | ADJ: [7, 9]
0-1-2-3 | STATUS: 0 | ADJ: [0, 1, 2, 3]
4-5 | STATUS: 0 | ADJ: [4, 5]
6-7-8-9-10 | STATUS: 0 | ADJ: [6, 7, 8, 9, 10]
end | STATUS: 0 | ADJ: []
TIME --------------------------------- 1
INFECTED: 3 6  RECOVERED: 
VIABLE SAMPLE LOCATIONS: 0-1-2-3 6-7-8-9-10 
TIME --------------------------------- 2
INFECTED: 3 8  RECOVERED: 6 
VIABLE SAMPLE LOCATIONS: 0-1-2-3 4-5 6-7-8-9-10 end 
TIME --------------------------------- 3
INFECTED: 3 8  RECOVERED: 6 
VIABLE SAMPLE LOCATIONS: 0-1-2-3 4-5 6-7-8-9-10 end 
TIME --------------------------------- 4
INFECTED: 3 8  RECOVERED: 6 
VIABLE SAMPLE LOCATIONS: 0-1-2-3 4-5 6-7-8-9-10 end 
TIME --------------------------------- 5
INFECTED: 8  RECOVERED: 6 

([<__main__.Node at 0x160f2676be0>, <__main__.Node at 0x160f26761c0>],
 [<__main__.Node at 0x160f2676b20>, <__main__.Node at 0x160f2659730>])

In [23]:
for i in sewer.keys():
    print(sewer[i])
    print(sewer[i].nearest_manhole)

0-1-2-3 | STATUS: 1 | ADJ: [0, 1, 2, 3]
4-5
4-5 | STATUS: 0 | ADJ: [4, 5]
6-7-8-9-10
6-7-8-9-10 | STATUS: 1 | ADJ: [6, 7, 8, 9, 10]
end
end | STATUS: 1 | ADJ: []
None
